In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, "../../code-previous")

In [4]:
import pandas as pd
import numpy as np
import operator

import peakutils

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import operator


import pickle
from sklearn.feature_extraction.text import CountVectorizer

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

In [7]:
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/word-journey-data.pql"
#data_grouped_by_year_label.to_pickle(filename)


data_grouped_by_year_label = pd.read_pickle(filename)

In [25]:
word = "tomography"
label = "Radiology, Nuclear Medicine & Medical Imaging"

In [8]:
data_grouped_by_year_label.head()

,index,Label,PubYear,title_without_stopwords,cleaned_titles,title_words_,title_words
0,0,Acoustics,1929,"[hearing speech auditoriums, hearing speech au...",[psychological measurements annoyance related ...,psychological measurements annoyance related p...,psychological measurements annoyance related p...
1,1,Acoustics,1930,[reverberation time bridge instrumental metho...,"[theory horn type loud speaker, discussion sym...",theory horn type loud speaker discussion sympo...,theory horn type loud speaker discussion sympo...
2,2,Acoustics,1931,"[some physical characteristics speech music, s...","[microphone technique radio broadcasting, orga...",microphone technique radio broadcasting organ ...,microphone technique radio broadcasting organ ...
3,3,Acoustics,1932,"[position finding under water sound signals, p...",[acoustic pick up philadelphia orchestra broad...,acoustic pick up philadelphia orchestra broadc...,acoustic pick up philadelphia orchestra broadc...
4,4,Acoustics,1933,"[some notes character bell tones, some notes c...","[sound absorption non reactive mixtures, deter...",sound absorption non reactive mixtures determi...,sound absorption non reactive mixtures determi...


In [15]:
data = data_grouped_by_year_label[(data_grouped_by_year_label.Label == label) &
                                  (data_grouped_by_year_label.PubYear > 2001) &
                                  (data_grouped_by_year_label.PubYear < 2012)]

In [16]:
len(data)

10

In [22]:
titles_ = data.cleaned_titles.tolist()

In [23]:
titles = [item for sublist in titles_ for item in sublist]

In [26]:
# filter titles which contain 
filtered_titles = [title for title in titles if word in title.split()]

In [27]:
len(filtered_titles)

5407

In [28]:
filtered_titles[:10]

['serendipitous detection cushing s disease positron emission tomography review literature',
 'finite difference time doniain  fdtd  analysis optical pulse responses biological tissues spectroscopic diffused optical tomography',
 'pulsed microwave induced thermoacoustic tomography  filtered backprojection circular measurement configuration',
 'noncontrast conventional computed tomography evaluation acute stroke',
 'positron emission tomography imaging melanoma lymphoma',
 'occupancy dopamine receptors mouse brain measured using ultra high resolution single photon emission tomography  i 123 ibf',
 'vivo human retinal imaging fourier domain optical coherence tomography',
 'comparison dobutamine echocardiography tc 99m sestamibi tomography prediction left ventricular ejection fraction outcome acute myocardial infarction treated successful primary coronary angioplasty',
 'detection hypervascular hepatocellular carcinoma dynamic magnetic resonance imaging double echo chemical shift in phase

In [36]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 10000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(filtered_titles)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(filtered_titles)
tf_feature_names = tf_vectorizer.get_feature_names()

In [38]:
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning:

n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21



In [39]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic:", topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("==========================================")

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic: 0
coronary angiography artery disease slice multislice computed 64 patients diagnostic
Topic: 1
positron emission 18 fluorodeoxyglucose cancer fluoro deoxy glucose imaging patients
Topic: 2
optical coherence using vivo imaging domain diffuse tissue polarization spectral
Topic: 3
magnetic resonance imaging findings computed comparison cardiac electrical features ultrasound
Topic: 4
beam cone computed using radiotherapy image electron dimensional prostate accuracy
Topic: 5
multidetector computed pulmonary evaluation row using assessment dimensional comparison left
Topic: 6
single photon emission myocardial perfusion computed 99m brain gated tc
Topic: 7
resolution high findings computed lung pulmonary pneumonia patients lungs pathologic
Topic: 8
dose radiation image computed quality low reduction cardiac therapy patient
Topic: 9
contrast enhanced dynamic computed liver comparison phase ultrasound using enhancement
Topic: 0
computed imaging magnetic resonance left findings cardiac v